https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/
Hyper parameter optimization:
https://stats.stackexchange.com/questions/65128/nested-cross-validation-for-model-selection
https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/

Current validation stack:
for each cancer and target and model
do 80/20 random stratified splits 200 times.
Feature select and then train on 80% and test on 20%


Nested cross validation:
do an 80/20 stratified split x times
within the 80:
do another 80/20 split y times
do this using your different possible hyper parameter models

Choose the one with the best outer loop cross validation performance
apply inner procedure to entire data set and use found hyper parameter for final model
then train on entire data set

https://scikit-learn.org/stable/modules/grid_search.html

In [ ]:
import pandas as pd
import processor as pr
import loader as load
import predictor as pred
import config

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, average_precision_score, log_loss, mean_squared_error

data, files = load.loadAll(includeStage=True, sameSamples=True, skipGenes=False, skipAE=True)

In [ ]:
ge = data[1]
stage_data = load.attachStageStatus(ge)
x, y = pr.splitData(stage_data, target="stage", project="STAD")

In [ ]:
y.value_counts()

In [ ]:
# https://aicorespot.io/how-to-produce-elastic-net-regression-models-in-python/

estimator = ElasticNet(random_state=0)
estimator.get_params()

param_grid = {
    "alpha" : [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0],
    "l1_ratio" : [step/10 for step in range(0, 10)]
}
# takes 2 minutes for 2 iterations
# for i in range(config.random_sampling_iterations):
for i in range(2):
    inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=i)
    x_inner, x_test, y_inner, y_test = train_test_split(x, y, test_size=(1 - config.random_sampling_training_portion), stratify=y, random_state=42+i)

    grid_search = GridSearchCV(
        estimator=estimator,
        param_grid=param_grid,
        cv=inner_cv,
        scoring="neg_root_mean_squared_error", #https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
        refit=True,
        return_train_score=True,
        n_jobs=6)

    grid_search.fit(x_inner, y_inner)
    
    # Predict on the test set and call accuracy
    y_pred_grid = grid_search.predict(x_test)
    metric_grid = mean_squared_error(y_test, y_pred_grid, squared=False)

    
    print("best parameters", grid_search.best_params_)
    print("best estimator", grid_search.best_estimator_)
    print("best score", grid_search.best_score_)
    # print("best cv results", grid_search.cv_results_)
    results_d = pd.DataFrame(grid_search.cv_results_)[['mean_test_score', 'std_test_score', 'params']]
    
    print(results_d)
    print("best_inner_model current generalization eval:", metric_grid)